# Task 1 a) Process Quadrat in MP4 Video
This notebook is configured to read a video specified by `file`, then extract the lines that compose the quadrat and estimate the appropriate corner points for cropping its 
contents. The output is a lower resolution MP4 video that is annotated with the found lines 
and corner points.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from glob import glob
from utils import draw_lines

# https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
#%load_ext autoreload
#%autoreload 2

figsize = (8, 6)
save_figures = False

In [ ]:
file = 'GLNI_12-1_2016-07-11_video-1.mp4'
#file = 'GLNI_456-3_2015-07-17_video-1.mp4'

In [ ]:
DATA_PATH = r'/scratch/gallowaa/cciw/Data'

# Search for all video files on Google Drive...
all_videos = glob(os.path.join(DATA_PATH, 'Videos_and_stills/GLNI/*/*/*/Videos/Quad*/*.mp4'))
videotable_path = os.path.join(DATA_PATH, 'Tables', 'QuadratVideos.csv')
video_df = pd.read_csv(videotable_path, index_col=0)

vpath = video_df.iloc[video_df[video_df['Name'] == file].index]['Quadrat Video Path']
tokens = video_df[video_df['Name'] == file]['Quadrat Video Path'].values[0].split('\\')

video_path = DATA_PATH + '/Videos_and_stills/GLNI'
for tok in tokens[4:-1]:
    video_path += '/' + tok
video_path = os.path.join(video_path, file)
print('Loading video: ', video_path)

In [ ]:
"""These are meta-parameters of the Probabilistic Hough Line Transform, 
note there are additional params in cell 6 which we set according to 
the input resolution.

@param rho Distance resolution of the accumulator (pixels)"""
rho = 1  

"""
@param theta Angle resolution of the accumulator (radians)

Suggest to use a value no less than np.pi/90 else too many 
spurious lines will be found. theta=np.pi/45 means lines 
must differ by 4 degrees to be considered distinct."""
theta = np.pi / 45

In [ ]:
cap = cv2.VideoCapture(video_path)
sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
      int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print('Raw input resolution', sz)

In [ ]:
# read first frame to adjust resolution of output stream
ret, im = cap.read() 

# set additional meta-parameters according to input res
if sz[0] == 1440:
    """x_trim and y_trim are used to remove black padding 
    which triggers spurious edges"""
    x_trim, y_trim = 1, 145
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    crop_frame_border = True
    
    """@param canny_thresh# hysteresis values for Canny edge 
    detector, input to HoughLines"""
    canny_thresh1, canny_thresh2 = 60, 300
    
    """@param threshold Accumulator threshold, return 
    lines with more than threshold of votes. (intersection points)"""
    threshold = 125
    
    """@param minLineLength Minimum line length. 
    Line segments shorter than that are rejected. (pixels)"""
    mLL = 400
    
    """@param maxLineGap Maximum allowed gap between points 
    on the same line to link them. (pixels)"""
    mLG = 150
else:
    # params as described above
    canny_thresh1, canny_thresh2 = 30, 300
    threshold = 125
    mLG, mLL = 250, 600
    crop_frame_border = False

"""this method may downsample, so set the video writer 
resolution to the processed image resolution"""
img, edges = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1)    
sz = (img.shape[1], img.shape[0])
print(sz)

In [ ]:
if cap.isOpened():
    fps = 20
    vout = cv2.VideoWriter()
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    vout.open(file.split('.')[0] + '-quadrat-demo.mp4', fourcc, fps, sz, True)
    print('Opened stream for writing, output resolution is', sz)
else:
    print('cap is not open')

In [ ]:
currentFrame = 0

"""it can take 30s-1min to process entire video, 
can optionally process a small number of frames"""
#for i in range(200):

# to process whole video    
while(True):
    
    # Capture frame-by-frame
    ret, im = cap.read()
    if not ret: break

    '''
    # For saving still images
    name = 'pframe_pi45_' + str(currentFrame) + '.jpg'
    save_path = os.path.join(out_path, name)
    print ('Creating...' + name)
    '''
    
    if crop_frame_border:
        im = im[y_trim:-y_trim, x_trim:-x_trim, :]
    
    # Do processing
    img, _ = draw_lines(im, rho=rho, theta=theta, mll=mLL, 
                        mlg=mLG, threshold=threshold, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)
    
    """Save still image in jpeg format
    cv2.imwrite(save_path, img)"""
    
    """For annotating video
    @param org Bottom-left corner of the text string (default=50).
    @param org Bottom-left corner of the text string in the image (default=50).
    @param fontFace Font type, see #HersheyFonts (default=cv2.FONT_HERSHEY_PLAIN).
    @param fontScale Font scale factor that is multiplied 
                     by the font-specific base size (default=2).
    @param color Text color (default=(R=0, G=255, B=0)).
    @param thickness Thickness of the lines used to draw a text (default=1).
    @param lineType Line type. See #LineTypes (default=cv2.LINE_AA)."""
    cv2.putText(                 # x, y
        img, str(currentFrame), (50, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0,255,0), 1, cv2.LINE_AA)
    vout.write(img)    

    # increment frame counter
    currentFrame += 1

"""When everything done, release the 
capture to flush the video stream. 

Before re-running this cell, first do 
cells 5, 6, and 7 to re-open cap and 
vout, otherwise ret=False and this 
cell does nothing."""
cap.release()
vout.release()

# End Demo

What follows is additional code for debugging

In [ ]:
"""
# to seek into a specific frame
for _ in range(28):
    ret, im = cap.read()
"""    
"""
ret, im = cap.read()
if ret is not None:
    plt.imshow(im)
"""

In [ ]:
'''
if crop_frame_border:
    im = im[y_trim:-y_trim, x_trim:-x_trim, :]

img, edges = draw_lines(im, rho=1, theta=np.pi/45, mll=mLL,
                        mlg=mLG, threshold=125, ds=1, 
                        canny_1=canny_thresh1, canny_2=canny_thresh2)
'''                        

In [ ]:
#plt.figure(figsize=(14, 12))
#plt.imshow(img)
#plt.imshow(im[y_trim:-y_trim, x_trim:-x_trim, :])

In [ ]:
#plt.imshow(edges)

In [ ]:
"""it can be helpful to save particularly difficult
frames for processing in a different notebook"""
#cv2.imwrite('test.jpg', im)

In [ ]:
"""
#cv2.circle(img, (800, 400), 10, (255, 0, 0), thickness=2, lineType=8, shift=0)
plt.figure(figsize=figsize)
plt.imshow(img)
plt.tight_layout()
if save_figures:
    plt.savefig('img/' + outpath + '-Step-2.png')
plt.show()
"""